In [2]:
!git clone https://github.com/AleksL04/ML-Project.git
%cd ML-Project

Cloning into 'ML-Project'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 122 (delta 56), reused 106 (delta 40), pack-reused 0 (from 0)
Receiving objects: 100% (122/122), 48.52 KiB | 4.04 MiB/s, done.
Resolving deltas: 100% (56/56), done.
/content/ML-Project


In [2]:
!git pull

Already up to date.


In [4]:
%%capture
!pip install --upgrade gensim
!gdown --folder 1rl_TJMHtcP-S0fTHNUpr9oi8Ndz2Y5z4 -O project_data
!gdown --folder 1RmC_We2lfsJontwrxlc4dzqEY_mNy-UW -O word2vec_model

!pip install --upgrade scikeras
!pip install "scikit-learn<1.6"


In [3]:
ls -l

total 36
-rw-r--r-- 1 root root 28331 Dec 12 00:42 main_console.ipynb
drwxr-xr-x 2 root root  4096 Dec 12 00:42 model_code/
-rw-r--r-- 1 root root  1550 Dec 12 00:42 process_df.py


In [5]:
import pandas as pd
import numpy as np

In [6]:
train_df = pd.read_csv('project_data/train.csv')
valid_df = pd.read_csv('project_data/valid.csv')
test_df = pd.read_csv('project_data/test.csv')

In [7]:
from process_df import process_df

X_train = process_df(train_df)
y_train = train_df['label'].values

X_valid = process_df(valid_df)
y_valid = valid_df['label'].values

X_test = process_df(test_df)
y_test = test_df['label'].values

In [8]:
from model_code.LSTM_code import trainLSTM, train_CV_LSTM
from model_code.CNN_code import trainCNN, train_CV_CNN
from model_code.Blender_code import trainBlender
from model_code.Bayes_code import trainBayes, predictBayes

In [ ]:
train_CV_LSTM(X_train, y_train, X_valid, y_valid)
#train_CV_CNN(X_train, y_train, X_valid, y_valid)

In [9]:
LSTM_model = trainLSTM(X_train, y_train, X_valid, y_valid)

test_loss, test_acc = LSTM_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8998 - loss: 0.2389
Test Accuracy: 0.8954


In [10]:
CNN_model = trainCNN(X_train, y_train, X_valid, y_valid)

test_loss, test_acc = CNN_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8458 - loss: 0.3429
Test Accuracy: 0.8395


In [ ]:
Bayes_model, vectorizer = trainBayes(train_df, valid_df)

Validation Accuracy: 0.7527932960893855
Validation Loss: 0.5019577052166138


In [ ]:
val_pred_LSTM = LSTM_model.predict(X_test).flatten()
val_pred_CNN = CNN_model.predict(X_test).flatten()
_,_,val_pred_Bayes = predictBayes(Bayes_model, valid_df, vectorizer)

X_meta_train = np.column_stack([
    val_pred_LSTM,
    val_pred_CNN,
    val_pred_Bayes
])

y_meta_train = y_valid

In [ ]:
# val_pred_LSTM = LSTM_model.predict(X_valid)
# val_pred_CNN = CNN_model.predict(X_valid)
# val_pred_Bayes = predictBayes(Bayes_model, valid_df, vectorizer)
# print(val_pred_Bayes)
# print(val_pred_LSTM)
# X_meta_train = np.column_stack((val_pred_LSTM, val_pred_CNN))

# y_meta_train = y_valid

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


AttributeError: 'tuple' object has no attribute 'predict'

In [ ]:
Blender_model = trainBlender(X_meta_train, y_meta_train)

In [ ]:
test_pred_LSTM = LSTM_model.predict(X_test)
test_pred_CNN = CNN_model.predict(X_test)

X_meta_test = np.column_stack((test_pred_LSTM, test_pred_CNN))

test_loss, test_acc = Blender_model.evaluate(X_meta_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")